### Introduction

Introduct the core topics of this tutorial.

### Contents

Details of the topics.

#### 1 Topic-1

#### 2 Topic-2

In [3]:
import numpy as np
@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
class NeuralNetwork:
        
    
    def __init__(self, 
                 no_of_in_nodes, 
                 no_of_out_nodes, 
                 no_of_hidden_nodes,
                 learning_rate,
                 bias=None
                ):  
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        
        self.no_of_hidden_nodes = no_of_hidden_nodes
            
        self.learning_rate = learning_rate 
        self.bias = bias
        self.create_weight_matrices()
    
        
    
    def create_weight_matrices(self):
        """ A method to initialize the weight matrices of the neural 
        network with optional bias nodes"""
        
        bias_node = 1 if self.bias else 0
        
        rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes, 
                                       self.no_of_in_nodes + bias_node))
        rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.weights_hidden_out = X.rvs((self.no_of_out_nodes, 
                                        self.no_of_hidden_nodes + bias_node))
        
        
        
    def train(self, input_vector, target_vector):
        # input_vector and target_vector can be tuple, list or ndarray
        
        bias_node = 1 if self.bias else 0
        if self.bias:
            # adding bias node to the end of the inpuy_vector
            input_vector = np.concatenate( (input_vector, [self.bias]) )
                                    
            
        input_vector = np.array(input_vector, ndmin=2).T
        target_vector = np.array(target_vector, ndmin=2).T
        
        output_vector1 = np.dot(self.weights_in_hidden, input_vector)
        output_vector_hidden = activation_function(output_vector1)
        
        if self.bias:
            output_vector_hidden = np.concatenate( (output_vector_hidden, [[self.bias]]) )
        
        
        output_vector2 = np.dot(self.weights_hidden_out, output_vector_hidden)
        output_vector_network = activation_function(output_vector2)
        
        output_errors = target_vector - output_vector_network
        # update the weights:
        tmp = output_errors * output_vector_network * (1.0 - output_vector_network)     
        tmp = self.learning_rate  * np.dot(tmp, output_vector_hidden.T)
        self.weights_hidden_out += tmp
        # calculate hidden errors:
        hidden_errors = np.dot(self.weights_hidden_out.T, output_errors)
        # update the weights:
        tmp = hidden_errors * output_vector_hidden * (1.0 - output_vector_hidden)
        if self.bias:
            x = np.dot(tmp, input_vector.T)[:-1,:]     # ???? last element cut off, ???
        else:
            x = np.dot(tmp, input_vector.T)
        self.weights_in_hidden += self.learning_rate * x
        
       
    
    def run(self, input_vector):
        # input_vector can be tuple, list or ndarray
        
        if self.bias:
            # adding bias node to the end of the inpuy_vector
            input_vector = np.concatenate( (input_vector, [1]) )
        input_vector = np.array(input_vector, ndmin=2).T
        output_vector = np.dot(self.weights_in_hidden, input_vector)
        output_vector = activation_function(output_vector)
        
        if self.bias:
            output_vector = np.concatenate( (output_vector, [[1]]) )
            
        output_vector = np.dot(self.weights_hidden_out, output_vector)
        output_vector = activation_function(output_vector)
    
        return output_vector

In [4]:
class1 = [(3, 4), (4.2, 5.3), (4, 3), (6, 5), (4, 6), (3.7, 5.8),
          (3.2, 4.6), (5.2, 5.9), (5, 4), (7, 4), (3, 7), (4.3, 4.3) ] 
class2 = [(-3, -4), (-2, -3.5), (-1, -6), (-3, -4.3), (-4, -5.6), 
          (-3.2, -4.8), (-2.3, -4.3), (-2.7, -2.6), (-1.5, -3.6), 
          (-3.6, -5.6), (-4.5, -4.6), (-3.7, -5.8) ]
labeled_data = []
for el in class1:
    labeled_data.append( [el, [1, 0]])
for el in class2:
    labeled_data.append([el, [0, 1]])
  
np.random.shuffle(labeled_data)
print(labeled_data[:10])
data, labels = zip(*labeled_data)
labels = np.array(labels)
data = np.array(data)

[[(-3, -4), [0, 1]], [(4, 3), [1, 0]], [(-1.5, -3.6), [0, 1]], [(3.2, 4.6), [1, 0]], [(4.3, 4.3), [1, 0]], [(-1, -6), [0, 1]], [(-2.3, -4.3), [0, 1]], [(5, 4), [1, 0]], [(6, 5), [1, 0]], [(7, 4), [1, 0]]]


In [5]:
simple_network = NeuralNetwork(no_of_in_nodes=2, 
                               no_of_out_nodes=2, 
                               no_of_hidden_nodes=10,
                               learning_rate=0.1,
                               bias=None)
    
for _ in range(20):
    for i in range(len(data)):
        simple_network.train(data[i], labels[i])
for i in range(len(data)):
    print(labels[i])
    print(simple_network.run(data[i]))

[0 1]
[[0.06299157]
 [0.93901625]]
[1 0]
[[0.89716614]
 [0.10315351]]
[0 1]
[[0.06944914]
 [0.93208909]]
[1 0]
[[0.90139844]
 [0.09953514]]
[1 0]
[[0.90195385]
 [0.09877472]]
[0 1]
[[0.06320614]
 [0.93835981]]
[0 1]
[[0.06350804]
 [0.93831154]]
[1 0]
[[0.90190626]
 [0.09872252]]
[1 0]
[[0.90337007]
 [0.0973981 ]]
[1 0]
[[0.90273858]
 [0.09790953]]
[0 1]
[[0.06148265]
 [0.9404527 ]]
[1 0]
[[0.89945017]
 [0.10147182]]
[1 0]
[[0.90320293]
 [0.09770495]]
[1 0]
[[0.90347658]
 [0.09748631]]
[0 1]
[[0.06238391]
 [0.93957695]]
[0 1]
[[0.06054654]
 [0.94139743]]
[0 1]
[[0.06742277]
 [0.93440439]]
[1 0]
[[0.9030733 ]
 [0.09777367]]
[0 1]
[[0.06054621]
 [0.94137817]]
[1 0]
[[0.90364451]
 [0.09719126]]
[0 1]
[[0.06098652]
 [0.94104694]]
[0 1]
[[0.07036944]
 [0.93215885]]
[0 1]
[[0.06066714]
 [0.94125802]]
[1 0]
[[0.90342287]
 [0.09746507]]


#### 3 Topic-3

### Summary
Summarize what we have covered in this tutorial.

### Further Reading

List resources that can help further understand the topics in this tutorial.